<a href="https://colab.research.google.com/github/sirius70/NLP_HW2/blob/main/reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Sentiment Labels

In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents.  As we discussed in class, many classification models represent a document as a sparse vector of of features and their weights. Weights could be boolean, counts, tf-idf values, or other functions of the document. In a linear model, we then combine the document vectors with the candidate output classes and learn weights for each of the feature-class pairs. This last step is implemented for you.

For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

There are many helpful resources online for getting up to speed with vector representations of documents. One example is the first two chapters of Delip Rao and Brian McMahan, _Natural Language Processing with PyTorch_, O'Reilly, 2019.  You should be able to <a href="https://learning.oreilly.com/library/view/natural-language-processing/9781491978221/">read it online</a> via the Northeastern Library's subscription using a <tt>northeastern.edu</tt> email address.

In [1]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [3]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here

  texts = [doc["text"] for doc in corpus] # all reviews
  classes = [doc["class"] for doc in corpus] # pos/neg labels

  # CountVectorizer with regex: words containing at least one alphabet
  def preprocess(text):
    return text.replace("_", " ")

  # Only keep tokens with alphabetic chars, after splitting underscores
  token_pattern = r'\b\w*[a-zA-Z]\w*\b'
  vectorizer = CountVectorizer(
      #preprocessor=preprocess,
      token_pattern=token_pattern
      )
  X = vectorizer.fit_transform(texts) # sparse matrix
  vocab = vectorizer.get_feature_names_out().tolist()

  return X,classes,vocab

In [4]:
# TODO: Implement createFancyFeatures andn describe in comments what the
# features are and why they might ben helpful.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createFancyFeatures(corpus):
  #Your code here

  texts = [doc["text"] for doc in corpus] # review texts
  classes = [doc["class"] for doc in corpus] # pos/neg labels

  def preprocess(text):
    return text.replace("_", " ")

  token_pattern = r'\b\w*[a-zA-Z]\w*\b'

  vectorizer = TfidfVectorizer(
      #preprocessor=preprocess,
      token_pattern=token_pattern,
      ngram_range=(1,2) # unigrams + bigrams
      )
  X = vectorizer.fit_transform(texts) # sparse TF-IDF matrix
  vocab = vectorizer.get_feature_names_out().tolist()
  return X,classes,vocab

In [5]:
# given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  # create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))

  # determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  # determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [6]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [7]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.827000
The most informative terms for pos are: ['flaws', 'memorable', 'terrific', 'edge', 'sherri', 'excellent', 'perfectly', 'masterpiece', 'enjoyable', 'using', 'fun', 'overall', 'gas', 'solid', 'command', 'quite', 'follows', 'fantastic', 'different', 'liar']
The most informative terms for neg are: ['waste', 'mess', 'ridiculous', 'lame', 'worst', 'headed', 'awful', 'unfortunately', 'cheap', 'write', 'boring', 'superior', 'tedious', 'bad', 'jesse', 'terrible', 'poor', 'maybe', 'looks', 'jakob']
----------L2 Norm-----------
The model's average accuracy is 0.833000
The most informative terms for pos are: ['fun', 'great', 'back', 'quite', 'well', 'excellent', 'overall', 'seen', 'american', 'perfectly', 'yet', 'memorable', 'terrific', 'job', 'pulp', 'true', 'very', 'performances', 'solid', 'different']
The most informative terms for neg are: ['bad', 'unfortunately', 'worst', 'waste', 'nothing', 'only', 'script', 'boring', 'awf

Run the following to train and evaluate two models using extended features:

In [8]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.703500
The most informative terms for pos are: ['and', 'life', 'is', 'as', 'his', 'great', 'the', 'very', 'jackie', 'truman', 'mulan', 'of', 'preachy henderson', 'preceded', 'preaches', 'preachers sonny', 'preachers go', 'preachers duvall', 'preachers at', 'preachers and']
The most informative terms for neg are: ['bad', 'worst', 't', 'have', 'plot', 'no', 'boring', 'only', 'to', 'there', 'movie', 'this', 'batman', 'nothing', 'harry', 'preacher duvall', 'preachers go', 'preachers duvall', 'preachers at', 'preachers and']
----------L2 Norm-----------
The model's average accuracy is 0.806000
The most informative terms for pos are: ['and', 'is', 'the', 'his', 'as', 'life', 'of', 'great', 'truman', 'very', 'he', 'jackie', 'also', 'he is', 'well', 'war', 'family', 'mulan', 'most', 'a']
The most informative terms for neg are: ['bad', 't', 'have', 'worst', 'movie', 'plot', 'no', 'boring', 'this', 'to', 'there', 'only', 'stupid', 'w

**TODO**: Briefly comment on your results. You do _not_ need to beat the basic features model to get full credit.